In [1]:
from algos import alternating_direction_method, fractional_step_method

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import math

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

In [3]:
def select_solver(scheme):
    solver = None
    if scheme == 0:
        solver = alternating_direction_method
    elif scheme == 1:
        solver = fractional_step_method
    else:
        raise ValueError

    return solver

In [4]:
y_begin_widget = widgets.Text('0.0')
y_end_widget = widgets.Text('1.0')

current_y_widget = widgets.FloatSlider(min=0.0, max=1.0, step=0.01)

def update_current_y_range(*args):
    try:
        begin = float(y_begin_widget.value)
        end = float(y_end_widget.value)
        current_y_widget.min = begin
        current_y_widget.max = end
        current_y_widget.step = (end - begin) / 100
    except Exception:
        print('error')
        
y_begin_widget.observe(update_current_y_range, 'value')
y_end_widget.observe(update_current_y_range, 'value')

x_begin_widget = widgets.Text('0.0')
x_end_widget = widgets.Text('1.0')

current_x_widget = widgets.FloatSlider(min=0.0, max=1.0, step=0.01)

def update_current_x_range(*args):
    try:
        begin = float(x_begin_widget.value)
        end = float(x_end_widget.value)
        current_x_widget.min = begin
        current_x_widget.max = end
        current_x_widget.step = (end - begin) / 100
    except Exception:
        print('error')
        
x_begin_widget.observe(update_current_x_range, 'value')
x_end_widget.observe(update_current_x_range, 'value')

t_begin_widget = widgets.Text('0.0')
t_end_widget = widgets.Text('1.0')

current_t_widget = widgets.FloatSlider(min=0.0, max=1.0, step=0.01)

def update_current_t_range(*args):
    try:
        begin = float(t_begin_widget.value)
        end = float(t_end_widget.value)
        current_t_widget.min = begin
        current_t_widget.max = end
        current_t_widget.step = (end - begin) / 100
    except Exception:
        print('error')
        
t_begin_widget.observe(update_current_t_range, 'value')
t_end_widget.observe(update_current_t_range, 'value')

def solve(scheme, a, x_begin, x_end, y_begin, y_end, t_begin, t_end, h1, h2, tau, current_x, current_y, current_t):
    solver = select_solver(scheme) 
    try:
        a = float(a) 

        x_begin = float(x_begin)
        x_end = float(x_end)

        y_begin = float(y_begin)
        y_end = float(y_end)
        
        t_begin = float(t_begin)
        t_end = float(t_end)
        
        h1 = float(h1)
        h2 = float(h2)
        tau = float(tau)
        
    except Exception:
        print('Please, recheck your input')
        return

    def f(x, y, t):
        return - x * y * np.sin(t)

    def phi1(y, t):
        return 0

    def phi2(y, t):
        return 0

    def phi3(x, t):
        return 0

    def phi4(x, t):
        return 0
    
    def psi(x, y):
        return x * y

    def u_ref(x, y, t):
        return x * y * np.cos(t)
    
    nx = round((x_end - x_begin) / h1) + 1
    ny = round((y_end - y_begin) / h2) + 1
    nt = round((t_end - t_begin) / tau) + 1

    x = np.linspace(x_begin, x_end, nx)
    y = np.linspace(y_begin, y_end, ny)
    t = np.linspace(t_begin, t_end, nt)
    
    u = solver(a, x_begin, x_end, y_begin, y_end, t_begin, t_end, phi1, phi2, phi3, phi4, psi, f, h1, h2, tau)

    n1 = math.floor((current_x - x_begin) / h1)
    n2 = math.floor((current_y - y_begin) / h2)
    n = math.floor((current_t - t_begin) / tau)
    
    u_ref_vals_x = u_ref(x, y[n2], t[n])
    u_ref_vals_y = u_ref(x[n1], y, t[n])
    
    fig, ax = plt.subplots()
    ax.plot(x, u[n, :, n2], 'g', label='result')
    ax.plot(x, u_ref_vals_x, 'r', label='reference')
    ax.legend()

    ax.set(xlabel='x', ylabel='u',
        title='u(x, y_fix, t_fix)')
    ax.grid()

    plt.show()
    
    fig, ax = plt.subplots()
    ax.plot(x, np.abs(u[n, :, n2] - u_ref_vals_x), 'r', label='error')
    ax.legend()

    ax.set(xlabel='x', ylabel='error',
        title='error(x)')
    ax.grid()

    plt.show()
    
    fig, ax = plt.subplots()
    ax.plot(y, u[n, n1], 'g', label='result')
    ax.plot(y, u_ref_vals_y, 'r', label='reference')
    ax.legend()

    ax.set(xlabel='y', ylabel='u',
        title='u(x_fix, y, t_fix)')
    ax.grid()

    plt.show()
    
    
    fig, ax = plt.subplots()
    ax.plot(y, np.abs(u[n, n1] - u_ref_vals_y), 'r', label='error')
    ax.legend()

    ax.set(xlabel='y', ylabel='error',
        title='error(y)')
    ax.grid()

    plt.show()
    
    hxs = []
    hys = []
    taus = []
    
    n_0 = 5
    n_n = 35
    
    for i in range(n_0, n_n):
        cur_hx = (x_end - x_begin) / (i - 1)
        hxs.append(cur_hx)
        cur_hy = (y_end - y_begin) / (i - 1)
        hys.append(cur_hy)
        cur_tau = (t_end - y_begin) / (i - 1)
        taus.append(cur_tau)

    hxs.reverse()
    hys.reverse()
    taus.reverse()
    
    errs = []
    for hx, hy, tau in zip(hxs, hys, taus):
        max_err = 0.0
        u = solver(a, x_begin, x_end, y_begin, y_end, t_begin, t_end, phi1, phi2, phi3, phi4, psi, f, hx, hy, tau)
            
        nx = round((x_end - x_begin) / hx) + 1
        ny = round((y_end - y_begin) / hy) + 1
        nt = round((t_end - t_begin) / tau) + 1
        
        x = np.linspace(x_begin, x_end, nx)
        y = np.linspace(y_begin, y_end, ny)

        for k in range(nt):
            for j in range(ny):
                u_ref_vals = u_ref(x, y_begin + hy * j, t_begin + tau * k)
                err = np.max(np.absolute(u[k, :, j] - u_ref_vals))
                if err > max_err:
                    max_err = err
                
        errs.append(max_err)

    errs = np.array(errs)

    log_steps = np.log(hxs)
    log_errs = np.log(errs)
    
    fig, ax = plt.subplots()
    
    ax.plot(log_steps, log_errs, 'r', label='max_error')
    ax.legend()

    ax.set(xlabel='h1', ylabel='max_error',
        title='max_error(h2)')
    ax.grid()

    plt.show()
    
    approximation_order = (log_errs[-1] - log_errs[0]) / (log_steps[-1] - log_steps[0])
    print(f'{approximation_order=}')
    
    
interact_manual(solve, scheme=[('Метод переменных направлений', 0), ('Метод дробных шагов', 1)],
                 a='1', x_begin=x_begin_widget, x_end=x_end_widget, y_begin=y_begin_widget, y_end=y_end_widget,
                 t_begin='0.0', t_end='1.0', h1='0.01', h2='0.02', tau='0.03', current_x=current_x_widget, current_y=current_y_widget, current_t=current_t_widget)
print()

interactive(children=(Dropdown(description='scheme', options=(('Метод переменных направлений', 0), ('Метод дро…